# Day 06: News-Based Trading Signals & Event Detection

## Week 19: NLP & Alternative Data

---

### Learning Objectives
1. **News Data Processing**: Acquire, clean, and structure financial news data
2. **Sentiment Analysis**: Apply NLP models to score news sentiment
3. **Event Detection**: Identify market-moving events using NER and classification
4. **Signal Generation**: Convert news sentiment into actionable trading signals
5. **Backtesting**: Evaluate news-based strategies with realistic assumptions

### Topics Covered
- Financial news data sources and APIs
- Text preprocessing for financial text
- Sentiment analysis with VADER, TextBlob, and FinBERT
- Named Entity Recognition (NER) for financial entities
- Event classification (earnings, M&A, regulatory)
- News momentum and decay effects
- Event study methodology
- Interview preparation questions

---

**Key Insight**: News-based trading exploits the information asymmetry between when news is published and when markets fully price in the information. Success depends on speed of processing, accuracy of sentiment analysis, and proper handling of noise.

## 1. Import Required Libraries

In [ ]:
# Core Libraries
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# NLP Libraries
import re
import string
from collections import Counter

# NLTK for text processing
import nltk
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('vader_lexicon', quiet=True)
    nltk.download('averaged_perceptron_tagger', quiet=True)

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# TextBlob for sentiment
try:
    from textblob import TextBlob
except ImportError:
    print("Install textblob: pip install textblob")

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-whitegrid')

# Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Statistical Analysis
from scipy import stats

print("✅ Core libraries loaded successfully!")
print(f"📅 Current Date: {datetime.now().strftime('%Y-%m-%d')}")

In [ ]:
# Optional: Transformers for advanced NLP (FinBERT)
try:
    from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
    import torch
    TRANSFORMERS_AVAILABLE = True
    print("✅ Transformers library available for FinBERT sentiment analysis")
except ImportError:
    TRANSFORMERS_AVAILABLE = False
    print("⚠️ Transformers not installed. Install with: pip install transformers torch")

# Optional: spaCy for NER
try:
    import spacy
    try:
        nlp = spacy.load("en_core_web_sm")
        SPACY_AVAILABLE = True
        print("✅ spaCy loaded with en_core_web_sm model")
    except OSError:
        SPACY_AVAILABLE = False
        print("⚠️ spaCy model not found. Run: python -m spacy download en_core_web_sm")
except ImportError:
    SPACY_AVAILABLE = False
    print("⚠️ spaCy not installed. Install with: pip install spacy")

## 2. Load and Preprocess News Data

### 2.1 Understanding News Data Sources

**Common Financial News Data Sources:**
- **Premium APIs**: Bloomberg, Reuters, Dow Jones, RavenPack
- **Free/Affordable**: Alpha Vantage News, Finnhub, NewsAPI, GDELT
- **Social/Alternative**: Twitter/X, Reddit, StockTwits
- **SEC Filings**: 8-K filings for material events

**Key Data Fields:**
- `timestamp`: Publication time (critical for trading)
- `headline`: Short title (most important for quick signals)
- `summary`: Brief description
- `body`: Full article text
- `source`: News provider
- `tickers`: Associated stock symbols
- `categories`: News type (earnings, M&A, etc.)

In [ ]:
# Create synthetic financial news dataset for demonstration
np.random.seed(42)

# Sample financial news headlines with various sentiment patterns
news_data = {
    'timestamp': pd.date_range(start='2025-01-01', periods=100, freq='4H'),
    'ticker': np.random.choice(['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA'], 100),
    'headline': [
        # Positive news
        "Apple Reports Record-Breaking Q4 Earnings, Beats Estimates",
        "Microsoft Cloud Revenue Surges 30% in Quarterly Results",
        "Google Announces Revolutionary AI Partnership Worth $10B",
        "Amazon Prime Membership Hits All-Time High",
        "Tesla Deliveries Exceed Analyst Expectations by 15%",
        "Meta's VR Division Shows Strong User Growth",
        "NVIDIA Secures Major Government AI Contract",
        "Apple Stock Upgraded to Strong Buy by Goldman Sachs",
        "Microsoft Azure Wins Massive Pentagon Cloud Deal",
        "Google Search Market Share Reaches New Peak",
        # Negative news
        "Apple Faces Major Supply Chain Disruptions in China",
        "Microsoft Layoffs Expand to 10,000 Employees",
        "Google Hit with $5 Billion Antitrust Fine in EU",
        "Amazon Workers Strike Affects Holiday Deliveries",
        "Tesla Recalls 500,000 Vehicles Over Safety Concerns",
        "Meta Faces Regulatory Scrutiny Over Privacy Violations",
        "NVIDIA GPU Shortage Worsens Amid Demand Surge",
        "Apple iPhone Sales Decline for Third Consecutive Quarter",
        "Microsoft Earnings Miss Analyst Expectations",
        "Google Advertising Revenue Growth Slows Significantly",
        # Neutral/Mixed news
        "Apple Plans New Product Launch Event Next Month",
        "Microsoft CEO Discusses Future of Remote Work",
        "Google Reorganizes Cloud Division Leadership",
        "Amazon Opens New Distribution Center in Texas",
        "Tesla Updates Autopilot Software Features",
        "Meta Announces Changes to Content Moderation Policy",
        "NVIDIA Releases New Graphics Card Lineup",
        "Apple Expands Services to New Markets",
        "Microsoft Invests in Quantum Computing Research",
        "Google Maps Adds New Navigation Features",
    ] * 3 + [
        "Apple and Microsoft Announce Strategic Partnership",
        "Fed Decision Could Impact Tech Stocks",
        "Semiconductor Industry Faces New Challenges",
        "Tech Giants Report Mixed Quarterly Results",
        "Market Volatility Affects Tech Sector",
        "New Regulations May Impact Big Tech Companies",
        "AI Investment Boom Continues Across Tech Sector",
        "Cloud Computing Market Reaches $500B Milestone",
        "Cybersecurity Concerns Rise for Tech Companies",
        "Tech IPO Market Shows Signs of Recovery"
    ],
    'source': np.random.choice(['Reuters', 'Bloomberg', 'CNBC', 'WSJ', 'MarketWatch'], 100),
    'category': np.random.choice(['Earnings', 'M&A', 'Product', 'Legal', 'Market', 'Personnel'], 100)
}

news_df = pd.DataFrame(news_data)
news_df = news_df.sample(frac=1).reset_index(drop=True)  # Shuffle
news_df['news_id'] = range(len(news_df))

print(f"📰 Loaded {len(news_df)} news articles")
print(f"📅 Date range: {news_df['timestamp'].min()} to {news_df['timestamp'].max()}")
print(f"📊 Tickers covered: {news_df['ticker'].nunique()}")
news_df.head(10)

In [ ]:
# Generate synthetic price data aligned with news
def generate_price_data(tickers, start_date, periods):
    """Generate synthetic price data with news-driven movements."""
    price_data = []
    
    for ticker in tickers:
        # Random walk with drift
        returns = np.random.normal(0.0005, 0.02, periods)
        prices = 100 * np.cumprod(1 + returns)
        
        dates = pd.date_range(start=start_date, periods=periods, freq='H')
        
        for i, (date, price) in enumerate(zip(dates, prices)):
            price_data.append({
                'timestamp': date,
                'ticker': ticker,
                'price': price,
                'volume': np.random.randint(1000000, 10000000)
            })
    
    return pd.DataFrame(price_data)

# Generate hourly price data
tickers = news_df['ticker'].unique()
price_df = generate_price_data(tickers, '2025-01-01', 400)

print(f"💹 Generated {len(price_df)} price observations")
print(f"📈 Price data sample:")
price_df.head()

### 2.2 Handling News Timestamps and Market Hours

**Critical Considerations:**
- News published after market close affects next day's open
- Pre-market news (4am-9:30am ET) affects opening prices
- After-hours news (4pm-8pm ET) affects next day
- Weekend news accumulates until Monday open

**Timestamp Alignment Strategy:**
```
News Time          →  Effective Trading Time
Before 9:30am ET   →  Same day open (9:30am)
9:30am - 4:00pm ET →  Current trading session
4:00pm - Midnight  →  Next trading day open
Weekends/Holidays  →  Next trading day open
```

In [ ]:
def align_news_to_trading_time(news_timestamp):
    """
    Align news publication time to effective trading time.
    Assumes US Eastern Time and NYSE trading hours.
    """
    # Market hours (simplified - doesn't account for holidays)
    market_open = 9.5  # 9:30 AM
    market_close = 16  # 4:00 PM
    
    hour = news_timestamp.hour + news_timestamp.minute / 60
    day_of_week = news_timestamp.weekday()
    
    # Weekend news → Monday open
    if day_of_week >= 5:  # Saturday or Sunday
        days_until_monday = 7 - day_of_week
        next_monday = news_timestamp + timedelta(days=days_until_monday)
        return next_monday.replace(hour=9, minute=30, second=0, microsecond=0)
    
    # Pre-market news → Same day open
    if hour < market_open:
        return news_timestamp.replace(hour=9, minute=30, second=0, microsecond=0)
    
    # During market hours → Current time
    if market_open <= hour < market_close:
        return news_timestamp
    
    # After hours → Next day open
    next_day = news_timestamp + timedelta(days=1)
    if next_day.weekday() >= 5:  # If next day is weekend
        days_until_monday = 7 - next_day.weekday()
        next_day = next_day + timedelta(days=days_until_monday)
    return next_day.replace(hour=9, minute=30, second=0, microsecond=0)

# Apply alignment
news_df['effective_time'] = news_df['timestamp'].apply(align_news_to_trading_time)
news_df['news_delay_hours'] = (news_df['effective_time'] - news_df['timestamp']).dt.total_seconds() / 3600

print("📊 News timing analysis:")
print(f"Average delay to effective time: {news_df['news_delay_hours'].mean():.2f} hours")
print(f"Max delay (weekend news): {news_df['news_delay_hours'].max():.2f} hours")
news_df[['timestamp', 'effective_time', 'news_delay_hours', 'headline']].head()

## 3. Text Cleaning and Tokenization

### 3.1 Text Preprocessing Pipeline

Financial text preprocessing requires special handling:
- Preserve financial terminology (P/E, M&A, Q1, 10-K)
- Keep numbers and percentages (they carry meaning!)
- Handle ticker symbols ($AAPL, AAPL)
- Process company name variations

In [ ]:
class FinancialTextPreprocessor:
    """
    Specialized text preprocessor for financial news.
    Preserves financially relevant information.
    """
    
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        # Remove financially relevant words from stopwords
        financial_keep = {'up', 'down', 'above', 'below', 'over', 'under', 'high', 'low', 'more', 'less'}
        self.stop_words = self.stop_words - financial_keep
        
        # Financial terms to preserve
        self.financial_terms = {
            'etf', 'ipo', 'pe', 'eps', 'ebitda', 'gaap', 'sec', 'fda', 'ftc',
            'ceo', 'cfo', 'coo', 'fed', 'fomc', 'gdp', 'cpi', 'pmi', 'm&a',
            'q1', 'q2', 'q3', 'q4', 'fy', 'yoy', 'qoq', 'ytd', 'roi', 'roa'
        }
        
    def clean_text(self, text):
        """Basic text cleaning while preserving financial information."""
        if not isinstance(text, str):
            return ""
        
        # Convert to lowercase
        text = text.lower()
        
        # Remove URLs
        text = re.sub(r'http\S+|www\S+', '', text)
        
        # Remove HTML tags
        text = re.sub(r'<[^>]+>', '', text)
        
        # Normalize ticker symbols ($AAPL → AAPL)
        text = re.sub(r'\$([A-Za-z]+)', r'\1', text)
        
        # Preserve percentages and numbers with context
        text = re.sub(r'(\d+)%', r'\1 percent', text)
        text = re.sub(r'\$(\d+(?:\.\d+)?)\s*(million|billion|trillion)?', 
                      r'\1 \2 dollars', text)
        
        # Remove special characters but keep apostrophes and hyphens
        text = re.sub(r"[^\w\s\'\-]", ' ', text)
        
        # Normalize whitespace
        text = ' '.join(text.split())
        
        return text
    
    def tokenize(self, text):
        """Tokenize text with financial awareness."""
        cleaned = self.clean_text(text)
        tokens = word_tokenize(cleaned)
        
        # Filter tokens
        filtered = []
        for token in tokens:
            # Keep financial terms
            if token.lower() in self.financial_terms:
                filtered.append(token.lower())
            # Keep if not a stopword and has meaningful length
            elif token.lower() not in self.stop_words and len(token) > 2:
                filtered.append(token.lower())
            # Keep numbers
            elif token.replace('.', '').replace(',', '').isdigit():
                filtered.append(token)
        
        return filtered
    
    def get_ngrams(self, text, n=2):
        """Extract n-grams from text."""
        tokens = self.tokenize(text)
        return list(zip(*[tokens[i:] for i in range(n)]))

# Initialize preprocessor
preprocessor = FinancialTextPreprocessor()

# Test preprocessing
sample_headline = "Apple ($AAPL) Reports 15% Revenue Growth in Q4, Beats Wall Street Estimates by $2 Billion"
print(f"Original: {sample_headline}")
print(f"Cleaned: {preprocessor.clean_text(sample_headline)}")
print(f"Tokens: {preprocessor.tokenize(sample_headline)}")
print(f"Bigrams: {preprocessor.get_ngrams(sample_headline, 2)[:5]}")

In [ ]:
# Apply preprocessing to news dataset
news_df['cleaned_headline'] = news_df['headline'].apply(preprocessor.clean_text)
news_df['tokens'] = news_df['headline'].apply(preprocessor.tokenize)
news_df['token_count'] = news_df['tokens'].apply(len)

# Analyze token distribution
print("📊 Token Analysis:")
print(f"Average tokens per headline: {news_df['token_count'].mean():.1f}")
print(f"Max tokens: {news_df['token_count'].max()}")
print(f"Min tokens: {news_df['token_count'].min()}")

# Most common words across all headlines
all_tokens = [token for tokens in news_df['tokens'] for token in tokens]
word_freq = Counter(all_tokens)
print(f"\n🔤 Top 15 most common words:")
for word, count in word_freq.most_common(15):
    print(f"  {word}: {count}")

## 4. Sentiment Analysis on News Headlines

### 4.1 Sentiment Analysis Approaches

**Three Common Methods:**

| Method | Pros | Cons | Speed |
|--------|------|------|-------|
| **VADER** | Pre-built, good for social media | Not trained on financial text | ⚡ Very Fast |
| **TextBlob** | Simple API, polarity + subjectivity | Generic, not financial-specific | ⚡ Fast |
| **FinBERT** | Trained on financial text, most accurate | Slow, requires GPU for scale | 🐢 Slow |

**Key Insight**: For high-frequency news trading, speed matters. VADER/TextBlob for quick signals, FinBERT for overnight batch processing.

In [ ]:
class FinancialSentimentAnalyzer:
    """
    Multi-method sentiment analyzer optimized for financial text.
    """
    
    def __init__(self):
        # VADER sentiment analyzer
        self.vader = SentimentIntensityAnalyzer()
        
        # Enhance VADER lexicon with financial terms
        financial_lexicon = {
            'beat': 2.0, 'beats': 2.0, 'exceeded': 2.0, 'surged': 2.5,
            'soared': 2.5, 'jumped': 2.0, 'rallied': 2.0, 'upgraded': 2.0,
            'outperformed': 2.0, 'record': 1.5, 'growth': 1.5, 'profit': 1.5,
            'gain': 1.5, 'gains': 1.5, 'bullish': 2.5, 'optimistic': 2.0,
            
            'missed': -2.0, 'miss': -2.0, 'fell': -1.5, 'dropped': -1.5,
            'plunged': -2.5, 'crashed': -3.0, 'downgraded': -2.0,
            'underperformed': -2.0, 'loss': -2.0, 'losses': -2.0,
            'decline': -1.5, 'declined': -1.5, 'bearish': -2.5,
            'recall': -2.0, 'lawsuit': -1.5, 'fine': -1.5, 'fined': -1.5,
            'layoffs': -2.0, 'layoff': -2.0, 'bankruptcy': -3.0,
            'investigation': -1.5, 'fraud': -3.0, 'scandal': -2.5,
            
            'volatility': -0.5, 'uncertain': -0.5, 'mixed': 0.0
        }
        self.vader.lexicon.update(financial_lexicon)
        
    def vader_sentiment(self, text):
        """Get VADER sentiment scores."""
        scores = self.vader.polarity_scores(text)
        return {
            'compound': scores['compound'],
            'positive': scores['pos'],
            'negative': scores['neg'],
            'neutral': scores['neu']
        }
    
    def textblob_sentiment(self, text):
        """Get TextBlob sentiment scores."""
        blob = TextBlob(text)
        return {
            'polarity': blob.sentiment.polarity,  # -1 to 1
            'subjectivity': blob.sentiment.subjectivity  # 0 to 1
        }
    
    def classify_sentiment(self, compound_score, thresholds=(-0.15, 0.15)):
        """Classify sentiment based on compound score."""
        neg_thresh, pos_thresh = thresholds
        if compound_score >= pos_thresh:
            return 'positive'
        elif compound_score <= neg_thresh:
            return 'negative'
        else:
            return 'neutral'
    
    def analyze(self, text):
        """Complete sentiment analysis."""
        vader = self.vader_sentiment(text)
        textblob = self.textblob_sentiment(text)
        
        return {
            'vader_compound': vader['compound'],
            'vader_pos': vader['positive'],
            'vader_neg': vader['negative'],
            'textblob_polarity': textblob['polarity'],
            'textblob_subjectivity': textblob['subjectivity'],
            'sentiment_class': self.classify_sentiment(vader['compound'])
        }

# Initialize analyzer
sentiment_analyzer = FinancialSentimentAnalyzer()

# Test on sample headlines
test_headlines = [
    "Apple Reports Record-Breaking Q4 Earnings, Beats Estimates",
    "Tesla Recalls 500,000 Vehicles Over Safety Concerns",
    "Microsoft Announces New Product Launch Event"
]

print("🎯 Sentiment Analysis Examples:\n")
for headline in test_headlines:
    result = sentiment_analyzer.analyze(headline)
    print(f"Headline: {headline}")
    print(f"  VADER Compound: {result['vader_compound']:.3f}")
    print(f"  TextBlob Polarity: {result['textblob_polarity']:.3f}")
    print(f"  Classification: {result['sentiment_class'].upper()}")
    print()

In [ ]:
# Apply sentiment analysis to entire dataset
sentiment_results = news_df['headline'].apply(sentiment_analyzer.analyze)
sentiment_df = pd.DataFrame(sentiment_results.tolist())

# Merge with original data
news_df = pd.concat([news_df, sentiment_df], axis=1)

# Sentiment distribution
print("📊 Sentiment Distribution:")
print(news_df['sentiment_class'].value_counts())
print(f"\n📈 Average VADER Compound: {news_df['vader_compound'].mean():.3f}")
print(f"📈 Average TextBlob Polarity: {news_df['textblob_polarity'].mean():.3f}")

# Visualize sentiment distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Histogram of VADER compound scores
axes[0].hist(news_df['vader_compound'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].axvline(x=0, color='red', linestyle='--', label='Neutral')
axes[0].axvline(x=0.15, color='green', linestyle='--', alpha=0.5, label='Positive threshold')
axes[0].axvline(x=-0.15, color='orange', linestyle='--', alpha=0.5, label='Negative threshold')
axes[0].set_xlabel('VADER Compound Score')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Sentiment Scores')
axes[0].legend()

# Sentiment class pie chart
sentiment_counts = news_df['sentiment_class'].value_counts()
colors = {'positive': 'green', 'negative': 'red', 'neutral': 'gray'}
axes[1].pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%',
            colors=[colors[c] for c in sentiment_counts.index])
axes[1].set_title('Sentiment Classification')

# VADER vs TextBlob correlation
axes[2].scatter(news_df['vader_compound'], news_df['textblob_polarity'], alpha=0.5, color='purple')
axes[2].set_xlabel('VADER Compound')
axes[2].set_ylabel('TextBlob Polarity')
axes[2].set_title('VADER vs TextBlob Correlation')
correlation = news_df['vader_compound'].corr(news_df['textblob_polarity'])
axes[2].text(0.05, 0.95, f'Correlation: {correlation:.3f}', transform=axes[2].transAxes)

plt.tight_layout()
plt.show()

### 4.2 FinBERT for Financial Sentiment (Advanced)

FinBERT is a BERT model pre-trained on financial text, providing state-of-the-art accuracy for financial sentiment analysis.

In [ ]:
def finbert_sentiment(headlines, batch_size=16):
    """
    Analyze sentiment using FinBERT model.
    More accurate but slower than VADER/TextBlob.
    """
    if not TRANSFORMERS_AVAILABLE:
        print("⚠️ Transformers not available. Using VADER fallback.")
        return None
    
    try:
        # Load FinBERT model
        finbert = pipeline("sentiment-analysis", 
                          model="ProsusAI/finbert",
                          tokenizer="ProsusAI/finbert",
                          truncation=True,
                          max_length=512)
        
        results = []
        for i in range(0, len(headlines), batch_size):
            batch = headlines[i:i+batch_size].tolist()
            batch_results = finbert(batch)
            results.extend(batch_results)
        
        # Convert to scores
        sentiment_scores = []
        for result in results:
            label = result['label']
            score = result['score']
            
            if label == 'positive':
                sentiment_scores.append(score)
            elif label == 'negative':
                sentiment_scores.append(-score)
            else:
                sentiment_scores.append(0)
        
        return sentiment_scores
        
    except Exception as e:
        print(f"⚠️ FinBERT error: {e}")
        return None

# Example FinBERT usage (only run if transformers available)
if TRANSFORMERS_AVAILABLE:
    print("🤖 Running FinBERT on sample headlines...")
    sample_headlines = news_df['headline'].head(10)
    finbert_scores = finbert_sentiment(sample_headlines)
    
    if finbert_scores:
        for headline, score in zip(sample_headlines, finbert_scores):
            print(f"[{score:+.3f}] {headline[:60]}...")
else:
    print("⚠️ FinBERT not available. Skipping advanced sentiment analysis.")
    print("   Install with: pip install transformers torch")

## 5. Named Entity Recognition for Event Detection

NER helps identify key entities in news:
- **Organizations**: Company names, regulators (SEC, FDA)
- **People**: CEOs, executives, analysts
- **Monetary Values**: Deal sizes, earnings figures
- **Dates**: Event dates, guidance periods
- **Locations**: Geographic markets

In [ ]:
class FinancialEntityExtractor:
    """
    Extract financial entities from news text.
    Uses spaCy if available, otherwise regex-based extraction.
    """
    
    def __init__(self):
        self.use_spacy = SPACY_AVAILABLE
        if self.use_spacy:
            self.nlp = nlp
        
        # Regex patterns for fallback
        self.patterns = {
            'money': re.compile(r'\$\d+(?:\.\d+)?(?:\s*(?:million|billion|trillion|M|B|T))?', re.I),
            'percentage': re.compile(r'\d+(?:\.\d+)?%'),
            'ticker': re.compile(r'\b[A-Z]{2,5}\b'),
            'quarter': re.compile(r'\b(?:Q[1-4]|FY\d{2,4})\b', re.I),
            'date': re.compile(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2}(?:,?\s+\d{4})?\b', re.I)
        }
        
        # Known company mappings
        self.company_tickers = {
            'apple': 'AAPL', 'microsoft': 'MSFT', 'google': 'GOOGL',
            'alphabet': 'GOOGL', 'amazon': 'AMZN', 'tesla': 'TSLA',
            'meta': 'META', 'facebook': 'META', 'nvidia': 'NVDA'
        }
        
    def extract_entities_spacy(self, text):
        """Extract entities using spaCy NER."""
        doc = self.nlp(text)
        entities = {
            'organizations': [],
            'people': [],
            'money': [],
            'dates': [],
            'locations': []
        }
        
        for ent in doc.ents:
            if ent.label_ == 'ORG':
                entities['organizations'].append(ent.text)
            elif ent.label_ == 'PERSON':
                entities['people'].append(ent.text)
            elif ent.label_ == 'MONEY':
                entities['money'].append(ent.text)
            elif ent.label_ in ('DATE', 'TIME'):
                entities['dates'].append(ent.text)
            elif ent.label_ in ('GPE', 'LOC'):
                entities['locations'].append(ent.text)
        
        return entities
    
    def extract_entities_regex(self, text):
        """Extract entities using regex patterns."""
        entities = {
            'organizations': [],
            'money': self.patterns['money'].findall(text),
            'percentages': self.patterns['percentage'].findall(text),
            'tickers': self.patterns['ticker'].findall(text),
            'quarters': self.patterns['quarter'].findall(text)
        }
        
        # Find company names
        text_lower = text.lower()
        for company, ticker in self.company_tickers.items():
            if company in text_lower:
                entities['organizations'].append(company.title())
        
        return entities
    
    def extract(self, text):
        """Extract all entities from text."""
        if self.use_spacy:
            spacy_entities = self.extract_entities_spacy(text)
            regex_entities = self.extract_entities_regex(text)
            # Merge results
            spacy_entities['percentages'] = regex_entities['percentages']
            spacy_entities['quarters'] = regex_entities['quarters']
            return spacy_entities
        else:
            return self.extract_entities_regex(text)
    
    def identify_tickers(self, text):
        """Identify stock tickers mentioned in text."""
        tickers = set()
        
        # Direct ticker mentions
        text_upper = text.upper()
        known_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA']
        for ticker in known_tickers:
            if ticker in text_upper:
                tickers.add(ticker)
        
        # Company name to ticker mapping
        text_lower = text.lower()
        for company, ticker in self.company_tickers.items():
            if company in text_lower:
                tickers.add(ticker)
        
        return list(tickers)

# Initialize entity extractor
entity_extractor = FinancialEntityExtractor()

# Test NER
test_text = "Apple CEO Tim Cook announced $10 billion investment in Q4 2025. Microsoft responded with their own AI initiative worth $5.5 billion."
entities = entity_extractor.extract(test_text)

print("🔍 Named Entity Recognition Results:\n")
print(f"Text: {test_text}\n")
for entity_type, values in entities.items():
    if values:
        print(f"  {entity_type.upper()}: {values}")

## 6. Building a News Sentiment Scoring System

### 6.1 Aggregating Sentiment by Ticker and Time Window

Key considerations:
- **Time decay**: Recent news is more relevant than old news
- **Source weighting**: Reuters/Bloomberg > random blog
- **Volume weighting**: More coverage = more significant event
- **Sentiment momentum**: Change in sentiment direction

In [ ]:
class NewsSentimentAggregator:
    """
    Aggregate news sentiment into trading signals by ticker and time window.
    """
    
    def __init__(self, decay_halflife_hours=24, source_weights=None):
        self.decay_halflife = decay_halflife_hours
        self.source_weights = source_weights or {
            'Reuters': 1.0,
            'Bloomberg': 1.0,
            'WSJ': 0.9,
            'CNBC': 0.8,
            'MarketWatch': 0.7
        }
        
    def calculate_decay_weight(self, hours_old):
        """Exponential decay weight based on news age."""
        return np.exp(-np.log(2) * hours_old / self.decay_halflife)
    
    def get_source_weight(self, source):
        """Get credibility weight for news source."""
        return self.source_weights.get(source, 0.5)
    
    def aggregate_sentiment(self, news_df, ticker, reference_time, lookback_hours=48):
        """
        Aggregate sentiment for a ticker at a specific time.
        
        Returns:
            dict: Aggregated sentiment metrics
        """
        # Filter news for ticker within lookback window
        start_time = reference_time - timedelta(hours=lookback_hours)
        mask = (
            (news_df['ticker'] == ticker) &
            (news_df['timestamp'] >= start_time) &
            (news_df['timestamp'] <= reference_time)
        )
        relevant_news = news_df[mask].copy()
        
        if len(relevant_news) == 0:
            return {
                'aggregate_sentiment': 0,
                'sentiment_momentum': 0,
                'news_volume': 0,
                'sentiment_dispersion': 0
            }
        
        # Calculate weights
        relevant_news['hours_old'] = (reference_time - relevant_news['timestamp']).dt.total_seconds() / 3600
        relevant_news['decay_weight'] = relevant_news['hours_old'].apply(self.calculate_decay_weight)
        relevant_news['source_weight'] = relevant_news['source'].apply(self.get_source_weight)
        relevant_news['total_weight'] = relevant_news['decay_weight'] * relevant_news['source_weight']
        
        # Weighted sentiment
        weighted_sentiment = (
            (relevant_news['vader_compound'] * relevant_news['total_weight']).sum() /
            relevant_news['total_weight'].sum()
        )
        
        # Sentiment momentum (recent vs older news)
        recent_cutoff = lookback_hours / 2
        recent_mask = relevant_news['hours_old'] <= recent_cutoff
        if recent_mask.sum() > 0 and (~recent_mask).sum() > 0:
            recent_sentiment = relevant_news.loc[recent_mask, 'vader_compound'].mean()
            older_sentiment = relevant_news.loc[~recent_mask, 'vader_compound'].mean()
            momentum = recent_sentiment - older_sentiment
        else:
            momentum = 0
        
        return {
            'aggregate_sentiment': weighted_sentiment,
            'sentiment_momentum': momentum,
            'news_volume': len(relevant_news),
            'sentiment_dispersion': relevant_news['vader_compound'].std() if len(relevant_news) > 1 else 0,
            'avg_source_quality': relevant_news['source_weight'].mean()
        }

# Initialize aggregator
sentiment_aggregator = NewsSentimentAggregator(decay_halflife_hours=24)

# Example aggregation
reference_time = news_df['timestamp'].max()
test_ticker = 'AAPL'

agg_result = sentiment_aggregator.aggregate_sentiment(news_df, test_ticker, reference_time)

print(f"📊 Aggregated Sentiment for {test_ticker} at {reference_time}:")
for key, value in agg_result.items():
    print(f"  {key}: {value:.4f}")

In [ ]:
# Build sentiment time series for all tickers
def build_sentiment_timeseries(news_df, tickers, freq='D'):
    """
    Build a time series of aggregated sentiment for each ticker.
    """
    # Get date range
    start_date = news_df['timestamp'].min().normalize()
    end_date = news_df['timestamp'].max().normalize()
    dates = pd.date_range(start=start_date, end=end_date, freq=freq)
    
    sentiment_series = []
    
    for ticker in tickers:
        for date in dates:
            agg = sentiment_aggregator.aggregate_sentiment(news_df, ticker, date, lookback_hours=48)
            sentiment_series.append({
                'date': date,
                'ticker': ticker,
                **agg
            })
    
    return pd.DataFrame(sentiment_series)

# Build time series
tickers = news_df['ticker'].unique()
sentiment_ts = build_sentiment_timeseries(news_df, tickers, freq='D')

print(f"📈 Built sentiment time series with {len(sentiment_ts)} observations")
print(f"📅 Date range: {sentiment_ts['date'].min()} to {sentiment_ts['date'].max()}")

# Visualize sentiment over time for one ticker
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

ticker_data = sentiment_ts[sentiment_ts['ticker'] == 'AAPL']

# Sentiment over time
axes[0].plot(ticker_data['date'], ticker_data['aggregate_sentiment'], 
             color='steelblue', linewidth=2, label='Aggregate Sentiment')
axes[0].fill_between(ticker_data['date'], ticker_data['aggregate_sentiment'], 
                     alpha=0.3, color='steelblue')
axes[0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[0].set_ylabel('Sentiment Score')
axes[0].set_title('AAPL News Sentiment Over Time')
axes[0].legend()

# News volume
axes[1].bar(ticker_data['date'], ticker_data['news_volume'], 
            color='coral', alpha=0.7, label='News Volume')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Number of Articles')
axes[1].set_title('AAPL News Volume Over Time')
axes[1].legend()

plt.tight_layout()
plt.show()

## 7. Detecting Market-Moving Events

### 7.1 Event Classification

Not all news is equal. Market-moving events include:
- **Earnings announcements**: Beats/misses, guidance changes
- **M&A activity**: Acquisitions, mergers, divestitures
- **Regulatory events**: FDA approvals, SEC investigations, antitrust
- **Management changes**: CEO departures, board changes
- **Product launches**: New products, discontinuations
- **Macro events**: Fed decisions, economic data